# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    possible_returns = []
    for index, row in data.iterrows():
        data.loc[index, "Possible Returns"] = data.loc[index, "Price in 1 Year"] / P - 1
        possible_returns.append((data.loc[index, "Probability"] * data.loc[index, "Possible Returns"])) 
        expected_return = sum(possible_returns)
    return expected_return

data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})

0.07500000000000002

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:

def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    lov = []
    for index, row in data.iterrows():
        data.loc[index, "Possible Returns"] = data.loc[index, "Price in 1 Year"] / P - 1
        lov.append((data.loc[index, "Probability"] * (data.loc[index, "Possible Returns"] - exp_return(data, P))**2))
    
    var = sum(lov)
    vol = np.sqrt(var)
    return vol
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})

0.21301408404140795

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    standard_error = volatility(data, P) / np.sqrt(len(data))
    print(standard_error)
    lower_bound = exp_return(data, P) - 1.96 * standard_error
    upper_bound = exp_return(data, P) + 1.96 * standard_error
    return [lower_bound,upper_bound]

data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})

0.12298373876248846


[-0.16604812797447732, 0.3160481279744774]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    ans = rf + beta * (market_return - rf)
    return ans


-0.028

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [8]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)
    df = df[pd.to_numeric(df["RET"], errors="coerce").notna()]
    security_counts = df["PERMNO"].value_counts()
    df = df[df["PERMNO"].isin(security_counts[security_counts >= 500].index)]
    df = df[["PERMNO", "date", "CUSIP", "RET"]]
    return df


PERMNO        date     CUSIP        RET
9027      10065  1970-01-30  00621210  -0.144375
9028      10065  1970-02-27  00621210   0.105882
9029      10065  1970-03-31  00621210   0.017857
9030      10065  1970-04-30  00621210  -0.078947
9031      10065  1970-05-29  00621210  -0.020952
...         ...         ...       ...        ...
3393558   84129  2022-08-31  98970110   0.016315
3393559   84129  2022-09-30  98970110  -0.075777
3393560   84129  2022-10-31  98970110   0.021235
3393561   84129  2022-11-30  98970110   0.005583
3393562   84129  2022-12-30  98970110  -0.051332

[403316 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [48]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    grouped = data.groupby("CUSIP")
    loc = []
    lor = []
    for name, group in grouped:
        loc.append(name)
        group["RET"] = pd.to_numeric(group["RET"])
        lor.append(group["RET"].mean(axis=0))
    output = pd.DataFrame(data={"CUSIP": loc, "Average Return": lor})
    return output


CUSIP  Average Return
0    00036110        0.012326
1    00095710        0.011339
2    00105510        0.018398
3    00108M10        0.011101
4    00120410        0.012135
..        ...             ...
674  G6359F13        0.019803
675  G7665A10        0.013261
676  G8994E10        0.013812
677  H2717810        0.011593
678  N5946510        0.019437

[679 rows x 2 columns]

In [38]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    grouped = data.groupby("CUSIP")
    loc = []
    lor = []
    for name, group in grouped:
        loc.append(name)
        group["RET"] = pd.to_numeric(group["RET"])
        lor.append(group["RET"].std(axis=0))
    output = pd.DataFrame(data={"CUSIP": loc, "Volatility": lor})
    return output


CUSIP  Volatility
0    00036110    0.122475
1    00095710    0.089881
2    00105510    0.099828
3    00108M10    0.171732
4    00120410    0.051050
..        ...         ...
674  G6359F13    0.205250
675  G7665A10    0.129473
676  G8994E10    0.090320
677  H2717810    0.131243
678  N5946510    0.134147

[679 rows x 2 columns]

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [51]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    df = pd.merge(ret_data, vol_data, on = "CUSIP")
    print(df.head())
    return df["Volatility"].corr(df["Average Return"])


      CUSIP  Average Return  Volatility
0  00036110        0.012326    0.122475
1  00095710        0.011339    0.089881
2  00105510        0.018398    0.099828
3  00108M10        0.011101    0.171732
4  00120410        0.012135    0.051050


0.44837090727059614